In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import numpy as np
import keras
import os
from pathlib import Path


/kaggle/input/data-file/fra.txt


In [2]:

dirpath = "/kaggle/input/data-file/fra.txt"



In [3]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 50000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path =dirpath


In [4]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path,"r",encoding="utf-8") as f:
    lines=f.read().split("\n")
for line in lines[: min(num_samples,len(lines)-1)]:
    input_text,target_text,_=line.split("\t")
    target_text="\t" +target_text +"\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

num_encoder_tokens=str(len(input_characters))
num_decoder_tokens=str(len(target_characters))
print("unique input characters: "  +  str(len(input_characters)))
print("unique target characters: "  +  str(len(target_characters)))


unique input characters: 78
unique target characters: 101


In [5]:
lengths=[]
for txt in input_texts:
    lengths.append(len(txt))
    
    
max_encoder_seq_length=max(lengths)
max_encoder_seq_length

21

In [6]:
lengths2=[]
for txt in target_texts:
    lengths.append(len(txt))
    
    
max_decoder_seq_length =max(lengths)
max_decoder_seq_length

67

In [7]:
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 50000
Number of unique input tokens: 78
Number of unique output tokens: 101
Max sequence length for inputs: 21
Max sequence length for outputs: 67


In [8]:
input_tokens_index=dict([(char,i) for i,char in enumerate(input_characters)])
target_tokens_index=dict([(char,i) for i,char in enumerate(target_characters)])


In [9]:
input_tokens_index

{'m': 0,
 'h': 1,
 ':': 2,
 'u': 3,
 ',': 4,
 'g': 5,
 'O': 6,
 '&': 7,
 'W': 8,
 '4': 9,
 '"': 10,
 'A': 11,
 '6': 12,
 'P': 13,
 '7': 14,
 'Z': 15,
 'ï': 16,
 'E': 17,
 'U': 18,
 'D': 19,
 'x': 20,
 'f': 21,
 '?': 22,
 '%': 23,
 '!': 24,
 'V': 25,
 'X': 26,
 't': 27,
 'I': 28,
 'B': 29,
 'Y': 30,
 'F': 31,
 '.': 32,
 '’': 33,
 'C': 34,
 'H': 35,
 'M': 36,
 'z': 37,
 'e': 38,
 'd': 39,
 'T': 40,
 '2': 41,
 'r': 42,
 'é': 43,
 'i': 44,
 '/': 45,
 'l': 46,
 'L': 47,
 '9': 48,
 's': 49,
 'R': 50,
 'j': 51,
 'o': 52,
 'S': 53,
 '0': 54,
 'n': 55,
 'Q': 56,
 'K': 57,
 '$': 58,
 'c': 59,
 'p': 60,
 'N': 61,
 "'": 62,
 'v': 63,
 '1': 64,
 '3': 65,
 'b': 66,
 'k': 67,
 ' ': 68,
 '5': 69,
 '8': 70,
 '-': 71,
 'J': 72,
 'y': 73,
 'a': 74,
 'G': 75,
 'w': 76,
 'q': 77}

In [11]:

encoder_input_data = np.zeros(
    (50000, 292, 90),
    dtype="float32",
)
decoder_input_data = np.zeros(
    (50000, 351, 114),
    dtype="float32",
)
decoder_target_data = np.zeros(
    (50000, 53519, 114),
    dtype="float32",
)

In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_tokens_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_tokens_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_tokens_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_tokens_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_tokens_index[" "]] = 1.0
    decoder_target_data[i, t:, target_tokens_index[" "]] = 1.0

In [ ]:
encoder_input_data

In [ ]:
from tensorflow import keras
import numpy as np



# Model configuration
num_encoder_tokens = 90  # Unique input tokens
num_decoder_tokens = 114 # Unique output tokens
latent_dim = 256  # Dimensionality of the latent space
max_encoder_seq_length = 292  # Maximum length of input sequences
max_decoder_seq_length = 351 # Maximum length of output sequences

# Encoder
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]  # Keep only states

# Decoder
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the full model
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

# Summary of the model
model.summary()


In [ ]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s_model.keras")


In [ ]:
pwd

In [ ]:

model = keras.models.load_model(r"C:\Users\zbook 17 g3\s2s_model.keras")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

reverse_input_char_index = dict((i, char) for char, i in input_tokens_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_tokens_index.items())


def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq, verbose=0)

    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_tokens_index["\t"]] = 1.0


    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value, verbose=0
        )

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        states_value = [h, c]
    return decoded_sentence


In [ ]:
for seq_index in range(90):
  
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)
